**Meditation and Neural Activities:** *Replication & Classifier Development*

**Final Data Science Neuroscience Project**<br>
A replication of Brandmeyer & Delorme (2018), with data-driven techniques. Creating new logistic regression model for classifier of expert vs. non-expert in mediation.

**Yuyang Zhong**<br>
University of California, Berkeley<br>
December 9, 2019

**Cognitive Neuroscience**<br>
Jack L. Gallant, Ph.D., *Professor*<br>
Manon Ironside, *Graduate Student Instructor*

----

# Introduction

## Background
Meditation had been claim to have a lot of physical and mental effects for individuals who actively practice meditation on a regular basis. However, to psychologists and neuroscientists, researchers are more interested in how these effects show up. Past research had focused on whether there was significant changes in subjects' neural activities when engaged in meditation over a period of time. 

The current research that this project is based on, conducted by Brandmeyer & Delorme in 2016, focuses on whether there was a significant difference in depth and neural activity (measured through EEG) for those who practice meditation on a more frequent basis (expert) compared to those on a less frequent basis (non-expert). 

This project will dive into understanding the published data better, and see if a classifier could be built to label expert vs. non-expert based on neural activities during meditation. 

## Motivation & Significance
In the literature, probe into meditation had lead researchers to find evidence of a Default-mode Network, as well as differences in functional connectivity of brain activities (Berkovich-Ohana et al. 2016;, Garrison et al. 2015). 

The present research (Brandmeyer & Delorme 2016), as well as this project, can potentially provide evidence for whether meditation alters one's default-mode network and functional connectivity, and attributing those change in neural activities to the benefits claimed by individuals practicing meditation as part of their daily lives. Authors of this paper were also trying to probe whether default mode network was related to the frequency of mind wandering episodes during meditation, especially those not aware by the individual (Christoff et al. 2009).

# Method

## Dataset Overview
This dataset was made available by the authors of the present research (Brandmeyer & Delorme 2016), at multiple open data repositories. Version 2.0 of the data, published on November 19, 2018, was downloaded from **Zenodo** (https://doi.org/10.5281/zenodo.2536267).

### Description by Author
This meditation experiment contains 24 subjects. Subjects were meditating and were interrupted about every 2 minutes to indicate their level of concentration and mind wandering.

### Dataset organization
The dataset is organized in the BIDS format. The raw data contains the MATLAB code for session, sound files to the stimuli, folders for each subject, within with folders for each session the subject participated in. In the session folders the eeg measures and event files are provided.

## Methods & Techniques
(Section referenced and adopted from original research article.)

Data was collected via EEG, using a 64-channel BioSemi system and and BioSemi 10-20 head cap montage. There are a total of 64 channels (locations of measure), mapped by the `Biosemi64Alpha` montage (not part of the standard `mne` packages, direction to load included below). This measure has very well temporal resolution but poor spatial resolution.

A total of 24 participants were in this study. Participants were asked to meditate for 30-90 seconds, and interrupted to rate their mindfulness depth and mind wondering level. This project will solely focus on the onset of that interruption, and the period of meditation before that. 

# Data Analysis

## Outline of Analysis
The following table summarizes the techniques used in each section.

| Section | Methods | Motivation |
| ------- | ------- | ---------- |
| **Data Exploration** | Time frequency analysis | Compute Time-Frequency Representation (TFR) using Morlet wavelet, and seeing if I can identify concentrations of epochs to focus on. |
| **Data Exploration** | Topographic Mapping: All Evoked Response | Looking at average brain activities near an event. |
| **Data Exploration** | Event-related Spectral Perturbation (ERSP): Onset Evoked Response | This would be important to help us understand whether this question is actually valid to ask - is there a difference in onset evoked response, and activities before that between the 2 subject groups? | 
| **Data Cleaning** | - | The data will be shrunken down to an average of all evoked responses, 10 seconds before the onset and 5 after, for each individual subjects. This will be used for the classifier. | 
| **Data Analysis** | Correlation Matrix | Looking at which channels are most and least correlated with each other. |
| **Data Analysis** | Independent Component Analysis (ICA) & Principal Component Analysis (PCA) | Looking at which channels are most significant in contribution to the overall brain activity, and try to see if I can figure out why. This would help identify components to use for our model. |
| **Classifier** | Logistic Regression | The simpler method. | 
| **Classifier** | Neural Network | A bit fancier method to improve accuracy. |
| **Classifier** | Random Forest | A bit fancier method to improve accuracy. |

## Project Setup & Imports
### Project Dependencies
This project utilizes the following Python packages:
`numpy`,
`pandas`,
`matplotlib`,
`seaborn`,
`mne`, and
`sklearn`.

To install a package within this jupyter notebook, utilize the command `!pip install [package-name]`. The `!` will allow you to run command line prompt within this notebook.

### Importing Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import mne

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, FastICA
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

### Suppresses Warnings
This is used for exporting the final PDF file. Feel free to comment this out.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## Data Exploration on Subject 1 (Non-Expert) & 15 (Expert)

### Importing data
**Assuming your currently directory is in the main folder. <br>**
Add `../` in the file paths if your working directory is in the `notebooks` folder of this project.

In [ ]:
raw_fname1 = './rawdata/bidsexport/sub-001/ses-01/eeg/sub-001_ses-01_task-meditation_eeg.bdf'
raw_fname15 = './rawdata/bidsexport/sub-015/ses-01/eeg/sub-015_ses-01_task-meditation_eeg.bdf'

raw1 = mne.io.read_raw_bdf(raw_fname1, preload=True)
raw15 = mne.io.read_raw_bdf(raw_fname15, preload=True)

For the purpose of this project, we will remove all of the channels that are metadata of the subject/experiment. We are focusing only on the 64 channels measuring EEG activities.

In [ ]:
raw1.drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 
                   'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp'])
raw15.drop_channels(['EXG1', 'EXG2', 'EXG3', 'EXG4', 'EXG5', 'EXG6', 'EXG7', 'EXG8', 
                   'GSR1', 'GSR2', 'Erg1', 'Erg2', 'Resp', 'Plet', 'Temp'])

### Loading & setting custom montage `biosemi64alpha`

Since the researchers used an Alphabetical (A/B) version of the standard `biosemi64` montage, we will need to load our own montage file to allow appropriate topographical mapping. 

In [ ]:
from os.path import abspath
montage = mne.channels.read_montage(abspath("biosemi64alpha.txt"))

raw1.set_montage(montage);
raw15.set_montage(montage);

### Subject Information
Let's print 1 subject's EEG information.

In [ ]:
print(raw1)
print(raw1.info)

The location of the sensors/channels are shown below. As you can see, the labels of the channels begin with A & B as aligned with the left/right hemispheres, instead of the specific naming of the standard BioSemi64 channel names.

In [ ]:
mne.viz.plot_sensors(raw1.info, ch_type='eeg', show_names=True);

### Events & Epochs
What kind of events are there in this session? This prints the top 5 events found for subject 1.

In [ ]:
events1 = mne.find_events(raw1, stim_channel='Status')
events15 = mne.find_events(raw15, stim_channel='Status')
print(events1[:5])
print(events15[:5])

From the `task-meditation_events.json` file, we found the following ID corresponding the events.

In [ ]:
import json
with open("rawdata/bidsexport/task-meditation_events.json", "r") as events_file:
    events = json.load(events_file)

event_dict = {i: d for d, i in events['value']['Levels'].items()}
print(event_dict)

Since only 3 events were used in the dataset, we will manually load those using the printout above.

In [ ]:
event_dict = {'Response 1 (this may be a response to question 1, 2 or 3)': 2,
                 'Response 2 (this may be a response to question 1, 2 or 3)': 4,
                 'First question onset (most important marker)': 128}

Setting epochs, we are only interested in 10 sections before the onset and 5 seconds after.

In [ ]:
epochs1 = mne.Epochs(raw1, events1, event_id=event_dict, tmin=-10, tmax=5, preload=True)
epochs15 = mne.Epochs(raw15, events15, event_id=event_dict, tmin=-10, tmax=5, preload=True)

We will then select the 3 conditions we care about (the 3 events above) and equalize them. Then we will select epochs related to these conditions. 

Since we only care about the onset, we will visualize the `onset_epoch` at channel `A2`, which in located in the prefrontal cortex.

In [ ]:
conds = ['Response 1 (this may be a response to question 1, 2 or 3)',
                       'Response 2 (this may be a response to question 1, 2 or 3)',
                      'First question onset (most important marker)']
epochs1.equalize_event_counts(conds)
epochs15.equalize_event_counts(conds)

r1_epochs1 = epochs1['Response 1 (this may be a response to question 1, 2 or 3)']
r2_epochs1 = epochs1['Response 2 (this may be a response to question 1, 2 or 3)']
onset_epochs1 = epochs1['First question onset (most important marker)']
onset_epochs1.plot_image(picks=['A2']);

r1_epochs15 = epochs15['Response 1 (this may be a response to question 1, 2 or 3)']
r2_epochs15 = epochs15['Response 2 (this may be a response to question 1, 2 or 3)']
onset_epochs15 = epochs15['First question onset (most important marker)']
onset_epochs15.plot_image(picks=['A2']);

We can see that there was an interesting dip around the onset for our expert mediator (second plot), but some hill for the non-expert (first plot). It may be related to mind wandering episodes that the non-expert may be invoking executive functions in the prefrontal cortex. It also seems that there are a totally inverse relationship for this channel. 

Let's try another channel, `B17`, located by the parietal lobe.

In [ ]:
onset_epochs1.plot_image(picks=['B17']);
onset_epochs15.plot_image(picks=['B17']);

`B17` is located in parietal lobe, and many research had found that this region have activity correlated with the default-mode network and overall functional connectivity of the brain. As you can see, the 2 subjects actually display similar pattern for the meditation, but a clear spike of activity in the non-expert (first plot) around the first second right after the onset of interruption. We can also see a greater variability in activities before the onset. Does this tell us about the more "smooth" brain activity, even in response to interruption, for expert meditators?

### Time Frequency Analysis
Since channel `B17` seem to be showing some interesting stuff, let's run a time frequency analysis for both subjects at this channel.

In [ ]:
frequencies = np.arange(7, 40, 1)
power1 = mne.time_frequency.tfr_morlet(onset_epochs1, n_cycles=3, return_itc=False,
                                      freqs=frequencies, decim=3)
power15 = mne.time_frequency.tfr_morlet(onset_epochs15, n_cycles=3, return_itc=False,
                                      freqs=frequencies, decim=3)
power1.plot(['B17']);
power15.plot(['B17']);

Well, it doesn't tell us much, but does confirm what we had discussed earlier. It seems that there's more variability in activity for the non-expert (first plot) than the expert (second plot) at channel `B17`.

### Evoked Response

Since this is an event related, we can take a look at the evoked response between the 2 subjects around the onset of the interruption. We only looked at 2 individual channels earlier, but we can also look at the aggregated response from all channels for these 2 subjects

In [ ]:
onset_evoked1 = onset_epochs1.average()
onset_evoked15 = onset_epochs15.average()

mne.viz.plot_compare_evokeds(dict(Subj1=onset_evoked1, Subj15=onset_evoked15),
                             legend='upper left', show_sensors='upper right');

It seems that even from the aggregated results we can tell which person is an expert and who is non-expert. There is a huge amount of variability for Subject 1 (blue line), while Subject 15's activities seemed to be relatively smooth overall. The spike around 3 seconds after onset (presumably when questions kick in) are drastically different as well.

We can also look at which brain region has the most activity around onset epochs.

In [ ]:
title1 = 'EEG Average reference (Subject 1)'
onset_evoked1.plot(titles=dict(eeg=title), time_unit='s')
onset_evoked1.plot_topomap(times=[-5,1,4], size=3., title=title1, time_unit='s');

title15 = 'EEG Average reference (Subject 15)'
onset_evoked15.plot(titles=dict(eeg=title), time_unit='s')
onset_evoked15.plot_topomap(times=[-5,1,4], size=3., title=title15, time_unit='s');

That's a very drastic differences for activities between these 2 subjects! There seems to be more frontal lobe activities for subject 1 (non-expert) and more parietal and occipital lobes activities for subject 15 (expert). Subject 1, again, shows a lot more variance compared to subject 15 in averaged EEG references.

Given all of the results found here around the onset epochs, we are more motivated to see whether this could be trained as a classifier to label the level of experience for a meditator. With this graph, it's quite convincing that it will work.

## Data Cleaning
For simplicity of the final output, data cleaning is performed via the notebook `data_cleaning.ipynb` in the same `notebooks` folder. A `*.csv` export of the clean up data will be used and loaded here. 

This is going to take a bit load. Good time to a tea break. Come back in a few minute.

In [ ]:
cleaned = pd.read_csv('final_data.csv').rename(columns={'Unnamed: 0': 'Time'})

## Data Analysis
### Separating Data: X, Y, Train, Validation, Test
We will be separating the data into 80% of epochs as training set, 20% of epochs as test set. We will use Cross Validation within as we train, using another 80-20 split. 

In [ ]:
# Setting Random Seed
np.random.seed(45)

# Partitioning X matrix and y vector
X = cleaned.drop('expert', axis=1)
y = cleaned['expert']

# Splitting into Train and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

### Correlation Matrix
Let's take a look at which channels are most correlated with each other.

In [ ]:
# Create correlation matrix
corr_matrix = X_train.corr().abs()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr_matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

It seems that a lot of these regions are loosely correlated with each other, but not a lot. There are several channels that are completely not related (lighter areas), say `B17` and `A2`, which were the 2 we discussed earlier, being in completely different regions of the brain (parietal vs. frontal).

### Independent Component Analysis (ICA)
Let's run ICA on our `X_train` and see what features stand out.

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
ica = FastICA(n_components=10) # 10 independent components
X_train_transformed = ica.fit_transform(X_train)

This plot will visualize what channels contributed to which ICs the most.

In [ ]:
plt.figure(figsize=(16,2))
plt.matshow(ica.components_, cmap='viridis', fignum=1, aspect='auto')
labs = []
plt.yticks(range(ica.components_.shape[0]), 
           [f'Comp# {i}' for i in range(1, ica.components_.shape[0]+11)],fontsize=10)
plt.colorbar()
plt.xticks(range(ica.components_.shape[1]), np.array(X.columns),rotation=90,ha='left')
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('axes', labelsize=10)
plt.show();

This ICA component matrix tells us that there are several channels of higher importances, even though with a very limited amount of variances explained. It does seem that time has something to do with the components as well.

### Principal Component Analysis (PCA)
We will try running PCA and see if any matching channels would show up from our results.

In [ ]:
pca = PCA(n_components=.95) # 95% variance explained
X_train_transformed = pca.fit_transform(X_train)

Similarly, this plot will visualize what channels contributed to which PCs the most.

In [ ]:
plt.figure(figsize=(16,6))
plt.matshow(pca.components_, cmap='viridis', fignum=1, aspect='auto')
labs = []
plt.yticks(range(pca.components_.shape[0]), 
           [f'Comp# {i}' for i in range(1, pca.components_.shape[0]+1)],fontsize=10)
plt.colorbar()
plt.xticks(range(pca.components_.shape[1]), np.array(X.columns),rotation=90,ha='left')
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('axes', labelsize=10)
plt.show();

The PCA yielded a total of 33 principal components and it seems that there are quiet a few channels highly related to some PCs. It seems that time is a huge factor for PC #10 and #11. We can plot the following scree plot to see whether any PCs stand out extremely from everything else.

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('PCs')
plt.ylabel('Cumulative explained variance')
plt.title('Scree Plot for Principal Components, 95% Variance Explained')
plt.show()

It seems that the first few PCs explain up to 50% of the variance. However, that is probably not enough for our classifier

## Classifiers
____
<center><b>Please note the the following analyses will take quite some time to run. It's time for a meal break, I'd say! The last 2 methods will probably take 30 minutes to an hour to run.</b></center>

____

### Logistic Regression (L-BFGS) with 5-Fold Cross Validation
We can now try to run a simple logistic regression model to predict the labels in the y-vector.

In [ ]:
# Create Logistic Function
logistic = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=500)  

# Cross Validation
cross_val_score(logistic, X_train, y_train, cv=5)

Our logistic regression using L-BFGS solver yielded a ~54.2% accuracy for our model.

### Logistic Regression (LASSO) with 5-Fold Cross Validation
We will introduce a L1 penalty and see if our result improves.

In [ ]:
# Create Logistic Function with L1 Penalty
logistic_l1 = LogisticRegression(penalty='l1', solver='saga', multi_class='multinomial', max_iter=500)  

# Cross Validation
cross_val_score(logistic_l1, X_train, y_train, cv=5)

This yielded similar results of a ~54.2% accuracy.... It's bad, but at least it's slightly better than chance (50%).

### Neural Network: Multi-Layer Perceptron classifier
We will deploy an MLP classifier to see if our result will improve. This will take about 30 minutes to run.

In [ ]:
mlp = MLPClassifier(activation='logistic')
mlp.fit(X_train, y_train)

In [ ]:
mlp.score(X_train, y_train)

In [ ]:
mlp.score(X_test, y_test)

It appears that our neural network is really bad at overfitting and yielded something of 99% accuracy on our training set (!!!!!), but ~54% on our test set... That's not good!

Let's see if random forest performs a little better.

### Random Forest
We will now deploy a random forest classifier on our data to see how well we did. This will take about an hour to run.

In [ ]:
clf = RandomForestClassifier(n_estimators=20, criterion='entropy')
clf.fit(X_train, y_train)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)

Similar to our MLP Classifier, we also way overfit on the training set, reaching a whopping 99.99% accuracy. But we do see a slight increase on our test set accuracy, which is at 57.4%, best in all of our methods. 

Let's take a moment to see which channels were most important in building our random forest. We will select the top 10 features.

In [ ]:
feature_imp = pd.Series(clf.feature_importances_, 
                       index=X.columns).sort_values(ascending=False)[:10]

sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.show();

We can see that the time seem to be some sort of determinant factor in our analysis, but not sure what. We can capture these important features/channels and visualize them on the sensor plot below.

In [ ]:
imp_chs = list(np.delete(np.array(feature_imp.index), list(feature_imp.index).index('Unnamed: 0')))
mne.viz.plot_sensors(raw1.info, ch_type='eeg', show_names=imp_chs);

From this plot we can see that the important factors are located near the parietal and occipital lobes, which had been found to have some correlation with functional connectivity and default mode network in other related research. 

# Discussion
## Conclusion
This project was able to roughly follow how the original research was set up to do, but instead of focusing on evidence of mind wondering (maybe yes given the amount of variance in activities we see in a non-expert subject), this project focused on what regions are more significant attributing the state of meditation, and whether or not those regions could be used to label brain activities during meditation as experts or non-experts. 

This project went through a pretty complete data science life cycle by first exploring extensively with the data, then settling on what regions of interests and types of analyses to be performed. The data was cleaned and stripped down to the onset epochs we care about, and taken as differences at 0-second/the onset as reference. The analyses were ranked simpler to more complicated, and all perform around a 54%+ range in correctly labeling our test data. Our best model, the Random Forest classifier, though overfitted on the training set, was still able to outperform all other models in the test set, returning a 57.4% accuracy in labeling expert/non-expert from brain activities around the onset opochs alone. Given the limited scope of this project, as well as my limited neuroscience expertise, this is a decent result for a machine learning classification task. 

## Limitations & Future Research
There are several limitations to this project. One, this project completely neglects individual differences by removing subject and sessions and aggregating only the brain data. It also removed continuous time scale and center around each individual onset epoch, and most likely neglecting the effect of adaptation with such repeated tasks of interruption. Second, little feature selection was performed to improve our model. Some basic steps were taken to attempt to select features, but more could be done to solidify that. Third, as the author in the original paper noted, we can take more of the responses to questions (depth of meditation and mind wondering awareness) into account, and seeing whether there would be a difference in terms of subject's labeling of their own mental states.

# References
**Berkovich-Ohana, A., Harel, M., Hahamy, A., Arieli, A., Malach, R. (2016).** Data for default network reduced functional connectivity in meditators, negatively correlated with meditation expertise. *Data Brief* 8:910–914

**Brandmeyer, T. & Delorme, A. (2018).** Reduced mind wandering in experienced meditators and associated EEG correlates. *Exp Brain Res* 236: 2519.

**Christoff, K., Gordon, A.M., Smallwood, J., Smith, R., Schooler, J.W. (2009).** Experience sampling during fMRI reveals default network and executive system contributions to mind wandering. *Proc Natl Acad Sci* 106(21):8719–8724

**Delorme, A. (2018).** BIDS formatted EEG meditation experiment data (Version 2.0) [Data set]. *Zenodo*. http://doi.org/10.5281/zenodo.2536267

**Garrison, K.A., Zeffiro, T.A., Scheinost, D., Constable, R.T., Brewer, J.A.. (2015).** Meditation leads to reduced default mode network activity beyond an active task. *Cogn Affect Behav Neurosci* 15(3):712–720


# Footnotes/Comments

1. Many of these codes are adopted from the `mne` package's examples and tutorials, as well as from the documentation of `scikit-learn` and `seaborn`.
1. See Appendix A (`data_cleaning.ipynb`) in the `notebooks` folder for the data cleaning steps.
1. This project took about 20 hours from beginning to end, concentrated on 3-4 days over the span of a month. Since I had zero prior knowledge of working with brain data in general and EEG specifically, the bulk of the time (~10 hours) was me mindlessly learning how to actually process the EEG data using the `mne` package, and figuring out the custom montage from scratch. Running the more advanced